# Hierarchical Clustering

**Hierarchical clustering** refers to a class of clustering methods that seek to build a **hierarchy** of clusters, in which some clusters contain others. In this assignment, we will explore a top-down approach, recursively bipartitioning the data using k-means.

**Note to Amazon EC2 users**: To conserve memory, make sure to stop all the other notebooks before running this notebook.

## Import packages

The following code block will check if you have the correct version of GraphLab Create. Any version later than 1.8.5 will do. To upgrade, read [this page](https://turi.com/download/upgrade-graphlab-create.html).

In [1]:
import graphlab
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import time
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
%matplotlib inline

'''Check GraphLab Create version'''
from distutils.version import StrictVersion
assert (StrictVersion(graphlab.version) >= StrictVersion('1.8.5')), 'GraphLab Create must be version 1.8.5 or later.'

## Load the Wikipedia dataset

In [2]:
wiki = graphlab.SFrame('people_wiki.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to superpalhares@gmail.com and will expire on November 01, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\super\AppData\Local\Temp\graphlab_server_1485828068.log.0


As we did in previous assignments, let's extract the TF-IDF features:

In [3]:
wiki['tf_idf'] = graphlab.text_analytics.tf_idf(wiki['text'])

To run k-means on this dataset, we should convert the data matrix into a sparse matrix.

In [4]:
from em_utilities import sframe_to_scipy # converter

# This will take about a minute or two.
tf_idf, map_index_to_word = sframe_to_scipy(wiki, 'tf_idf')

In [6]:
tf_idf

<59071x547979 sparse matrix of type '<type 'numpy.float64'>'
	with 10379283 stored elements in Compressed Sparse Row format>

To be consistent with the k-means assignment, let's normalize all vectors to have unit norm.

In [5]:
from sklearn.preprocessing import normalize
tf_idf = normalize(tf_idf)

## Bipartition the Wikipedia dataset using k-means

Recall our workflow for clustering text data with k-means:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with some value of k.
4. Visualize the clustering results using the centroids, cluster assignments, and the original dataframe. We keep the original dataframe around because the data matrix does not keep auxiliary information (in the case of the text dataset, the title of each article).

Let us modify the workflow to perform bipartitioning:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with k=2.
4. Divide the data matrix into two parts using the cluster assignments.
5. Divide the dataframe into two parts, again using the cluster assignments. This step is necessary to allow for visualization.
6. Visualize the bipartition of data.

We'd like to be able to repeat Steps 3-6 multiple times to produce a **hierarchy** of clusters such as the following:
```
                      (root)
                         |
            +------------+-------------+
            |                          |
         Cluster                    Cluster
     +------+-----+             +------+-----+
     |            |             |            |
   Cluster     Cluster       Cluster      Cluster
```
Each **parent cluster** is bipartitioned to produce two **child clusters**. At the very top is the **root cluster**, which consists of the entire dataset.

Now we write a wrapper function to bipartition a given cluster using k-means. There are three variables that together comprise the cluster:

* `dataframe`: a subset of the original dataframe that correspond to member rows of the cluster
* `matrix`: same set of rows, stored in sparse matrix format
* `centroid`: the centroid of the cluster (not applicable for the root cluster)

Rather than passing around the three variables separately, we package them into a Python dictionary. The wrapper function takes a single dictionary (representing a parent cluster) and returns two dictionaries (representing the child clusters).

In [6]:
def bipartition(cluster, maxiter=400, num_runs=4, seed=None):
    '''cluster: should be a dictionary containing the following keys
                * dataframe: original dataframe
                * matrix:    same data, in matrix format
                * centroid:  centroid for this particular cluster'''
    
    data_matrix = cluster['matrix']
    dataframe   = cluster['dataframe']
    
    # Run k-means on the data matrix with k=2. We use scikit-learn here to simplify workflow.
    kmeans_model = KMeans(n_clusters=2, max_iter=maxiter, n_init=num_runs, random_state=seed, n_jobs=1, verbose=1)
    print "Fiting"
    kmeans_model.fit(data_matrix)
    print "Fited"
    centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_
    
    # Divide the data matrix into two parts using the cluster assignments.
    data_matrix_left_child, data_matrix_right_child = data_matrix[cluster_assignment==0], \
                                                      data_matrix[cluster_assignment==1]
    
    # Divide the dataframe into two parts, again using the cluster assignments.
    cluster_assignment_sa = graphlab.SArray(cluster_assignment) # minor format conversion
    dataframe_left_child, dataframe_right_child     = dataframe[cluster_assignment_sa==0], \
                                                      dataframe[cluster_assignment_sa==1]
        
    
    # Package relevant variables for the child clusters
    cluster_left_child  = {'matrix': data_matrix_left_child,
                           'dataframe': dataframe_left_child,
                           'centroid': centroids[0]}
    cluster_right_child = {'matrix': data_matrix_right_child,
                           'dataframe': dataframe_right_child,
                           'centroid': centroids[1]}

    print 'End'
    return (cluster_left_child, cluster_right_child)

The following cell performs bipartitioning of the Wikipedia dataset. Allow 20-60 seconds to finish.

Note. For the purpose of the assignment, we set an explicit seed (`seed=1`) to produce identical outputs for every run. In pratical applications, you might want to use different random seeds for all runs.

In [7]:
wiki_data = {'matrix': tf_idf, 'dataframe': wiki} # no 'centroid' for the root cluster
left_child, right_child = bipartition(wiki_data, maxiter=100, num_runs=6, seed=1)

Fiting
Initialization complete
Iteration  0, inertia 116403.036
Iteration  1, inertia 58351.852
Iteration  2, inertia 58297.635
Iteration  3, inertia 58261.706
Iteration  4, inertia 58229.069
Iteration  5, inertia 58220.610
Iteration  6, inertia 58219.679
Iteration  7, inertia 58219.221
Iteration  8, inertia 58218.996
Iteration  9, inertia 58218.868
Iteration 10, inertia 58218.790
Iteration 11, inertia 58218.744
Iteration 12, inertia 58218.713
Iteration 13, inertia 58218.688
Iteration 14, inertia 58218.672
Iteration 15, inertia 58218.659
Iteration 16, inertia 58218.653
Iteration 17, inertia 58218.649
Iteration 18, inertia 58218.647
Iteration 19, inertia 58218.645
Iteration 20, inertia 58218.644
Iteration 21, inertia 58218.643
Iteration 22, inertia 58218.643
Iteration 23, inertia 58218.642
Iteration 24, inertia 58218.642
Converged at iteration 24: center shift 0.000000e+00 within tolerance 1.803739e-10
Initialization complete
Iteration  0, inertia 116703.253
Iteration  1, inertia 58287.

Let's examine the contents of one of the two clusters, which we call the `left_child`, referring to the tree visualization above.

In [8]:
left_child

{'centroid': array([  0.00000000e+00,   8.57526623e-06,   0.00000000e+00, ...,
          1.38560691e-04,   6.46049863e-05,   2.26551103e-05]),
 'dataframe': Columns:
 	URI	str
 	name	str
 	text	str
 	tf_idf	dict
 
 Rows: Unknown
 
 Data:
 +-------------------------------+-------------------------------+
 |              URI              |              name             |
 +-------------------------------+-------------------------------+
 | <http://dbpedia.org/resour... |         Digby Morrell         |
 | <http://dbpedia.org/resour... | Paddy Dunne (Gaelic footba... |
 | <http://dbpedia.org/resour... |         Ceiron Thomas         |
 | <http://dbpedia.org/resour... |          Adel Sellimi         |
 | <http://dbpedia.org/resour... |          Vic Stasiuk          |
 | <http://dbpedia.org/resour... |          Leon Hapgood         |
 | <http://dbpedia.org/resour... |           Dom Flora           |
 | <http://dbpedia.org/resour... |           Bob Reece           |
 | <http://dbpedia.org/re

And here is the content of the other cluster we named `right_child`.

In [9]:
right_child

{'centroid': array([  3.00882137e-06,   0.00000000e+00,   2.88868244e-06, ...,
          1.10291526e-04,   9.00609890e-05,   2.03703564e-05]),
 'dataframe': Columns:
 	URI	str
 	name	str
 	text	str
 	tf_idf	dict
 
 Rows: Unknown
 
 Data:
 +-------------------------------+---------------------+
 |              URI              |         name        |
 +-------------------------------+---------------------+
 | <http://dbpedia.org/resour... |    Alfred J. Lewy   |
 | <http://dbpedia.org/resour... |    Harpdog Brown    |
 | <http://dbpedia.org/resour... | Franz Rottensteiner |
 | <http://dbpedia.org/resour... |        G-Enka       |
 | <http://dbpedia.org/resour... |    Sam Henderson    |
 | <http://dbpedia.org/resour... |    Aaron LaCrate    |
 | <http://dbpedia.org/resour... |   Trevor Ferguson   |
 | <http://dbpedia.org/resour... |     Grant Nelson    |
 | <http://dbpedia.org/resour... |     Cathy Caruth    |
 | <http://dbpedia.org/resour... |     Sophie Crumb    |
 +-------------------

## Visualize the bipartition

We provide you with a modified version of the visualization function from the k-means assignment. For each cluster, we print the top 5 words with highest TF-IDF weights in the centroid and display excerpts for the 8 nearest neighbors of the centroid.

In [10]:
def display_single_tf_idf_cluster(cluster, map_index_to_word):
    '''map_index_to_word: SFrame specifying the mapping betweeen words and column indices'''
    
    wiki_subset   = cluster['dataframe']
    tf_idf_subset = cluster['matrix']
    centroid      = cluster['centroid']
    
    # Print top 5 words with largest TF-IDF weights in the cluster
    idx = centroid.argsort()[::-1]
    for i in xrange(5):
        print('{0:s}:{1:.3f}'.format(map_index_to_word['category'][idx[i]], centroid[idx[i]])),
    print('')
    
    # Compute distances from the centroid to all data points in the cluster.
    distances = pairwise_distances(tf_idf_subset, [centroid], metric='euclidean').flatten()
    # compute nearest neighbors of the centroid within the cluster.
    nearest_neighbors = distances.argsort()
    # For 8 nearest neighbors, print the title as well as first 180 characters of text.
    # Wrap the text at 80-character mark.
    for i in xrange(8):
        text = ' '.join(wiki_subset[nearest_neighbors[i]]['text'].split(None, 25)[0:25])
        print('* {0:50s} {1:.5f}\n  {2:s}\n  {3:s}'.format(wiki_subset[nearest_neighbors[i]]['name'],
              distances[nearest_neighbors[i]], text[:90], text[90:180] if len(text) > 90 else ''))
    print('')

Let's visualize the two child clusters:

In [11]:
display_single_tf_idf_cluster(left_child, map_index_to_word)

league:0.040 season:0.036 team:0.029 football:0.029 played:0.028 
* Todd Williams                                      0.95468
  todd michael williams born february 13 1971 in syracuse new york is a former major league 
  baseball relief pitcher he attended east syracuseminoa high school
* Gord Sherven                                       0.95622
  gordon r sherven born august 21 1963 in gravelbourg saskatchewan and raised in mankota sas
  katchewan is a retired canadian professional ice hockey forward who played
* Justin Knoedler                                    0.95639
  justin joseph knoedler born july 17 1980 in springfield illinois is a former major league 
  baseball catcherknoedler was originally drafted by the st louis cardinals
* Chris Day                                          0.95648
  christopher nicholas chris day born 28 july 1975 is an english professional footballer who
   plays as a goalkeeper for stevenageday started his career at tottenham
* Tony Smith (football

In [12]:
display_single_tf_idf_cluster(right_child, map_index_to_word)

she:0.025 her:0.017 music:0.012 he:0.011 university:0.011 
* Anita Kunz                                         0.97401
  anita e kunz oc born 1956 is a canadianborn artist and illustratorkunz has lived in london
   new york and toronto contributing to magazines and working
* Janet Jackson                                      0.97472
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Madonna (entertainer)                              0.97475
  madonna louise ciccone tkoni born august 16 1958 is an american singer songwriter actress 
  and businesswoman she achieved popularity by pushing the boundaries of lyrical
* %C3%81ine Hyland                                   0.97536
  ine hyland ne donlon is emeritus professor of education and former vicepresident of univer
  sity college cork ireland she was born in 1942 in athboy co
* Jane Fonda                                         0.9

The left cluster consists of athletes, whereas the right cluster consists of non-athletes. So far, we have a single-level hierarchy consisting of two clusters, as follows:

```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
                 Athletes                                      Non-athletes
```

Is this hierarchy good enough? **When building a hierarchy of clusters, we must keep our particular application in mind.** For instance, we might want to build a **directory** for Wikipedia articles. A good directory would let you quickly narrow down your search to a small set of related articles. The categories of athletes and non-athletes are too general to facilitate efficient search. For this reason, we decide to build another level into our hierarchy of clusters with the goal of getting more specific cluster structure at the lower level. To that end, we subdivide both the `athletes` and `non-athletes` clusters.

## Perform recursive bipartitioning

### Cluster of athletes

To help identify the clusters we've built so far, let's give them easy-to-read aliases:

In [13]:
athletes = left_child
non_athletes = right_child

Using the bipartition function, we produce two child clusters of the athlete cluster:

In [14]:
# Bipartition the cluster of athletes
left_child_athletes, right_child_athletes = bipartition(athletes, maxiter=100, num_runs=6, seed=1)

Fiting
Initialization complete
Iteration  0, inertia 21880.358
Iteration  1, inertia 11161.130
Iteration  2, inertia 11148.967
Iteration  3, inertia 11137.437
Iteration  4, inertia 11136.021
Iteration  5, inertia 11135.937
Iteration  6, inertia 11135.925
Iteration  7, inertia 11135.921
Converged at iteration 7: center shift 0.000000e+00 within tolerance 1.773880e-10
Initialization complete
Iteration  0, inertia 22082.265
Iteration  1, inertia 11156.365
Iteration  2, inertia 11119.674
Iteration  3, inertia 11112.811
Iteration  4, inertia 11112.159
Iteration  5, inertia 11111.968
Iteration  6, inertia 11111.919
Iteration  7, inertia 11111.901
Iteration  8, inertia 11111.893
Iteration  9, inertia 11111.888
Iteration 10, inertia 11111.885
Iteration 11, inertia 11111.883
Iteration 12, inertia 11111.882
Converged at iteration 12: center shift 0.000000e+00 within tolerance 1.773880e-10
Initialization complete
Iteration  0, inertia 21796.318
Iteration  1, inertia 11126.350
Iteration  2, inerti

The left child cluster mainly consists of baseball players:

In [15]:
display_single_tf_idf_cluster(left_child_athletes, map_index_to_word)

baseball:0.111 league:0.103 major:0.051 games:0.046 season:0.045 
* Steve Springer                                     0.89344
  steven michael springer born february 11 1961 is an american former professional baseball 
  player who appeared in major league baseball as a third baseman and
* Dave Ford                                          0.89598
  david alan ford born december 29 1956 is a former major league baseball pitcher for the ba
  ltimore orioles born in cleveland ohio ford attended lincolnwest
* Todd Williams                                      0.89823
  todd michael williams born february 13 1971 in syracuse new york is a former major league 
  baseball relief pitcher he attended east syracuseminoa high school
* Justin Knoedler                                    0.90097
  justin joseph knoedler born july 17 1980 in springfield illinois is a former major league 
  baseball catcherknoedler was originally drafted by the st louis cardinals
* Kevin Nicholson (baseball)        

On the other hand, the right child cluster is a mix of players in association football, Austrailian rules football and ice hockey:

In [16]:
display_single_tf_idf_cluster(right_child_athletes, map_index_to_word)

season:0.034 football:0.033 team:0.031 league:0.029 played:0.027 
* Gord Sherven                                       0.95562
  gordon r sherven born august 21 1963 in gravelbourg saskatchewan and raised in mankota sas
  katchewan is a retired canadian professional ice hockey forward who played
* Ashley Prescott                                    0.95656
  ashley prescott born 11 september 1972 is a former australian rules footballer he played w
  ith the richmond and fremantle football clubs in the afl between
* Chris Day                                          0.95656
  christopher nicholas chris day born 28 july 1975 is an english professional footballer who
   plays as a goalkeeper for stevenageday started his career at tottenham
* Jason Roberts (footballer)                         0.95658
  jason andre davis roberts mbe born 25 january 1978 is a former professional footballer and
   now a football punditborn in park royal london roberts was
* Todd Curley                         

Our hierarchy of clusters now looks like this:
```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
                 Athletes                                      Non-athletes
                    +
                    |
        +-----------+--------+
        |                    |
        |            association football/
        +          Austrailian rules football/
     baseball             ice hockey
```

Should we keep subdividing the clusters? If so, which cluster should we subdivide? To answer this question, we again think about our application. Since we organize our directory by topics, it would be nice to have topics that are about as coarse as each other. For instance, if one cluster is about baseball, we expect some other clusters about football, basketball, volleyball, and so forth. That is, **we would like to achieve similar level of granularity for all clusters.**

Notice that the right child cluster is more coarse than the left child cluster. The right cluster posseses a greater variety of topics than the left (ice hockey/association football/Austrialian football vs. baseball). So the right child cluster should be subdivided further to produce finer child clusters.

Let's give the clusters aliases as well:

In [17]:
baseball            = left_child_athletes
ice_hockey_football = right_child_athletes

### Cluster of ice hockey players and football players

In answering the following quiz question, take a look at the topics represented in the top documents (those closest to the centroid), as well as the list of words with highest TF-IDF weights.

Let us bipartition the cluster of ice hockey and football players.

In [18]:
left_child_ihs, right_child_ihs = bipartition(ice_hockey_football, maxiter=100, num_runs=6, seed=1)
display_single_tf_idf_cluster(left_child_ihs, map_index_to_word)
display_single_tf_idf_cluster(right_child_ihs, map_index_to_word)

Fiting
Initialization complete
Iteration  0, inertia 18838.009
Iteration  1, inertia 9450.606
Iteration  2, inertia 9431.201
Iteration  3, inertia 9415.277
Iteration  4, inertia 9412.805
Iteration  5, inertia 9412.659
Iteration  6, inertia 9412.625
Iteration  7, inertia 9412.592
Iteration  8, inertia 9412.545
Iteration  9, inertia 9412.447
Iteration 10, inertia 9412.257
Iteration 11, inertia 9412.011
Iteration 12, inertia 9411.855
Iteration 13, inertia 9411.487
Iteration 14, inertia 9408.346
Iteration 15, inertia 9405.823
Iteration 16, inertia 9405.797
Iteration 17, inertia 9405.781
Iteration 18, inertia 9405.773
Iteration 19, inertia 9405.769
Iteration 20, inertia 9405.768
Iteration 21, inertia 9405.768
Converged at iteration 21: center shift 0.000000e+00 within tolerance 1.774290e-10
Initialization complete
Iteration  0, inertia 18712.753
Iteration  1, inertia 9438.013
Iteration  2, inertia 9419.381
Iteration  3, inertia 9417.893
Iteration  4, inertia 9417.771
Iteration  5, inertia 9

**Quiz Question**. Which diagram best describes the hierarchy right after splitting the `ice_hockey_football` cluster? Refer to the quiz form for the diagrams.

**Caution**. The granularity criteria is an imperfect heuristic and must be taken with a grain of salt. It takes a lot of manual intervention to obtain a good hierarchy of clusters.

* **If a cluster is highly mixed, the top articles and words may not convey the full picture of the cluster.** Thus, we may be misled if we judge the purity of clusters solely by their top documents and words. 
* **Many interesting topics are hidden somewhere inside the clusters but do not appear in the visualization.** We may need to subdivide further to discover new topics. For instance, subdividing the `ice_hockey_football` cluster led to the appearance of runners and golfers.

### Cluster of non-athletes

Now let us subdivide the cluster of non-athletes.

In [19]:
# Bipartition the cluster of non-athletes
left_child_non_athletes, right_child_non_athletes = bipartition(non_athletes, maxiter=100, num_runs=6, seed=1)

Fiting
Initialization complete
Iteration  0, inertia 93155.499
Iteration  1, inertia 46859.208
Iteration  2, inertia 46846.249
Iteration  3, inertia 46845.145
Iteration  4, inertia 46844.880
Iteration  5, inertia 46844.818
Iteration  6, inertia 46844.797
Iteration  7, inertia 46844.784
Iteration  8, inertia 46844.777
Iteration  9, inertia 46844.771
Iteration 10, inertia 46844.767
Iteration 11, inertia 46844.764
Iteration 12, inertia 46844.762
Iteration 13, inertia 46844.761
Iteration 14, inertia 46844.761
Iteration 15, inertia 46844.761
Converged at iteration 15: center shift 0.000000e+00 within tolerance 1.803028e-10
Initialization complete
Iteration  0, inertia 93255.369
Iteration  1, inertia 46865.288
Iteration  2, inertia 46846.007
Iteration  3, inertia 46845.303
Iteration  4, inertia 46845.140
Iteration  5, inertia 46845.070
Iteration  6, inertia 46845.030
Iteration  7, inertia 46844.996
Iteration  8, inertia 46844.961
Iteration  9, inertia 46844.929
Iteration 10, inertia 46844.89

In [20]:
display_single_tf_idf_cluster(left_child_non_athletes, map_index_to_word)

he:0.013 music:0.012 university:0.011 film:0.010 his:0.009 
* Wilson McLean                                      0.97870
  wilson mclean born 1937 is a scottish illustrator and artist he has illustrated primarily 
  in the field of advertising but has also provided cover art
* Julian Knowles                                     0.97938
  julian knowles is an australian composer and performer specialising in new and emerging te
  chnologies his creative work spans the fields of composition for theatre dance
* James A. Joseph                                    0.98042
  james a joseph born 1935 is an american former diplomatjoseph is professor of the practice
   of public policy studies at duke university and founder of
* Barry Sullivan (lawyer)                            0.98054
  barry sullivan is a chicago lawyer and as of july 1 2009 the cooney conway chair in advoca
  cy at loyola university chicago school of law
* Archie Brown                                       0.98081
  archibal

In [21]:
display_single_tf_idf_cluster(right_child_non_athletes, map_index_to_word)

she:0.126 her:0.082 film:0.013 actress:0.012 music:0.012 
* Janet Jackson                                      0.93808
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Lauren Royal                                       0.93867
  lauren royal born march 3 circa 1965 is a book writer from california royal has written bo
  th historic and novelistic booksa selfproclaimed angels baseball fan
* Barbara Hershey                                    0.93941
  barbara hershey born barbara lynn herzstein february 5 1948 once known as barbara seagull 
  is an american actress in a career spanning nearly 50 years
* Jane Fonda                                         0.94102
  jane fonda born lady jayne seymour fonda december 21 1937 is an american actress writer po
  litical activist former fashion model and fitness guru she is
* Alexandra Potter                                   0.94190
  al

Neither of the clusters show clear topics, apart from the genders. Let us divide them further.

In [22]:
male_non_athletes = left_child_non_athletes
female_non_athletes = right_child_non_athletes

**Quiz Question**. Let us bipartition the clusters `male_non_athletes` and `female_non_athletes`. Which diagram best describes the resulting hierarchy of clusters for the non-athletes? Refer to the quiz for the diagrams.

**Note**. Use `maxiter=100, num_runs=6, seed=1` for consistency of output.

In [23]:
female_non_athletes_left, female_non_athletes_right = bipartition(female_non_athletes, maxiter=100, num_runs=6, seed=1)

Fiting
Initialization complete
Iteration  0, inertia 16575.280
Iteration  1, inertia 8467.506
Iteration  2, inertia 8462.583
Iteration  3, inertia 8461.786
Iteration  4, inertia 8461.274
Iteration  5, inertia 8460.949
Iteration  6, inertia 8460.733
Iteration  7, inertia 8460.589
Iteration  8, inertia 8460.513
Iteration  9, inertia 8460.468
Iteration 10, inertia 8460.428
Iteration 11, inertia 8460.399
Iteration 12, inertia 8460.380
Iteration 13, inertia 8460.373
Iteration 14, inertia 8460.369
Iteration 15, inertia 8460.365
Iteration 16, inertia 8460.363
Iteration 17, inertia 8460.361
Converged at iteration 17: center shift 0.000000e+00 within tolerance 1.759936e-10
Initialization complete
Iteration  0, inertia 16878.563
Iteration  1, inertia 8472.169
Iteration  2, inertia 8466.482
Iteration  3, inertia 8465.099
Iteration  4, inertia 8464.368
Iteration  5, inertia 8463.542
Iteration  6, inertia 8462.728
Iteration  7, inertia 8461.926
Iteration  8, inertia 8461.111
Iteration  9, inertia 8

In [24]:
male_non_athletes_left, male_non_athletes_right = bipartition(male_non_athletes, maxiter=100, num_runs=6, seed=1)

Fiting
Initialization complete
Iteration  0, inertia 76578.986
Iteration  1, inertia 38261.177
Iteration  2, inertia 38226.036
Iteration  3, inertia 38220.595
Iteration  4, inertia 38218.011
Iteration  5, inertia 38216.416
Iteration  6, inertia 38215.519
Iteration  7, inertia 38215.055
Iteration  8, inertia 38214.803
Iteration  9, inertia 38214.620
Iteration 10, inertia 38214.479
Iteration 11, inertia 38214.379
Iteration 12, inertia 38214.311
Iteration 13, inertia 38214.266
Iteration 14, inertia 38214.236
Iteration 15, inertia 38214.216
Iteration 16, inertia 38214.202
Iteration 17, inertia 38214.190
Iteration 18, inertia 38214.179
Iteration 19, inertia 38214.174
Iteration 20, inertia 38214.169
Iteration 21, inertia 38214.166
Iteration 22, inertia 38214.164
Iteration 23, inertia 38214.162
Iteration 24, inertia 38214.161
Iteration 25, inertia 38214.160
Iteration 26, inertia 38214.159
Iteration 27, inertia 38214.159
Iteration 28, inertia 38214.159
Iteration 29, inertia 38214.159
Iteration

In [25]:
display_single_tf_idf_cluster(male_non_athletes_left, map_index_to_word)
display_single_tf_idf_cluster(male_non_athletes_right, map_index_to_word)

university:0.017 he:0.015 law:0.013 served:0.013 research:0.013 
* Barry Sullivan (lawyer)                            0.97075
  barry sullivan is a chicago lawyer and as of july 1 2009 the cooney conway chair in advoca
  cy at loyola university chicago school of law
* James A. Joseph                                    0.97344
  james a joseph born 1935 is an american former diplomatjoseph is professor of the practice
   of public policy studies at duke university and founder of
* David Anderson (British Columbia politician)       0.97383
  david a anderson pc oc born august 16 1937 in victoria british columbia is a former canadi
  an cabinet minister educated at victoria college in victoria
* Sven Erik Holmes                                   0.97469
  sven erik holmes is a former federal judge and currently the vice chairman legal risk and 
  regulatory and chief legal officer for kpmg llp a
* Andrew Fois                                        0.97558
  andrew fois is an attorney livi

In [26]:
display_single_tf_idf_cluster(female_non_athletes_left, map_index_to_word)
display_single_tf_idf_cluster(female_non_athletes_right, map_index_to_word)

she:0.121 her:0.100 actress:0.031 film:0.030 music:0.028 
* Janet Jackson                                      0.92374
  janet damita jo jackson born may 16 1966 is an american singer songwriter and actress know
  n for a series of sonically innovative socially conscious and
* Barbara Hershey                                    0.92524
  barbara hershey born barbara lynn herzstein february 5 1948 once known as barbara seagull 
  is an american actress in a career spanning nearly 50 years
* Madonna (entertainer)                              0.92753
  madonna louise ciccone tkoni born august 16 1958 is an american singer songwriter actress 
  and businesswoman she achieved popularity by pushing the boundaries of lyrical
* Cher                                               0.92909
  cher r born cherilyn sarkisian may 20 1946 is an american singer actress and television ho
  st described as embodying female autonomy in a maledominated industry
* Candice Bergen                               